In [1]:
# Import libraries
from functions import *

import json
import os
import subprocess
import time

# Creating a transaction with multiple inputs and outputs

In this section we'll create a P2WPKH transaction with multiple inputs and outputs. We'll go through each part of the transaction, how it's constructed, signed, and we'll test it using bitcoin core in regtest mode.

This section assumes knowledge of creating a P2WPKH transaction with a single input.

## Reading
- Andreas Antonopoulos - Mastering Bitcoin Chapter 7
- Jimmy Song - Programming Bitcoin Chapters 13

### Scenario

Andreas and Lisa both want to support three charities, all in foreign countries. The charities accept bitcoin donations, so they decide to send bitcoin jointly in a single transaction. Using a single transaction has a couple of advantages over each of them creating their own transactions:
- The single transaction is smaller than two separate ones, saving Andreas and Lisa some btc on transaction fees.
- The charities benefit by receiving one UTXO rather than two making it cheaper to spend from. 

Andreas has a UTXO for 0.3 btc, of which he wants to donate all of it. Lisa has a UTXO for 0.401 and she wants to match Andreas's donation of 0.3 btc. They want to donate 0.2 btc to the following addresses:
`bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw`, `bcrt1q6mlqttg852e63uahyglwla55xusryqp08vx9w2`, `bcrt1qe9y40n9uwzh34mzj02w3xx9zkhgke6wxcql4lk`


### Create Andreas and Anita's P2WPKH UTXOs

We'll use bitcoind to create Andreas and Anita's P2WPKH UTXOs. Within this tutorial, to differentiate the variables for Andreas and Lisa by using the suffixes `_a` and `_l` respectively.

In [2]:
# Create two private keys and P2WPKH addresses
privkey_a = bytes.fromhex("1111111111111111111111111111111111111111111111111111111111111111")
pubkey_a = privkey_to_pubkey(privkey_a)
p2wpkh_addr_a = pk_to_p2wpkh(pubkey_a, network = "regtest")

privkey_l = bytes.fromhex("2222222222222222222222222222222222222222222222222222222222222222")
pubkey_l = privkey_to_pubkey(privkey_l)
p2wpkh_addr_l = pk_to_p2wpkh(pubkey_l, network = "regtest")

# Setup bitcoind and fund addresses
setup_regtest_bitcoind()
txid_a, index_a = fund_address(p2wpkh_addr_a, 0.3)
txid_l, index_l = fund_address(p2wpkh_addr_l, 0.401)

print(f"Andreas's UTXO: {txid_a}, {index_a}")
print(f"Lisa's UTXO: {txid_l}, {index_l}")

Andreas's UTXO: 96d9881a73ce8cbab9c342c5484c57b6183126c7882457ffc90fcfa2d38bcb46, 1
Lisa's UTXO: beee8ead0e38ab8fcdd54612ec54c5e311b4c18e2c4cc42d663ecbb9598a71c9, 0


### Define our outputs

Since Lisa is only donating 0.3 btc of her 0.401 btc utxo (and 0.001 btc is going towards the mining fee), Lisa wants to send the remaning 0.1 btc to her change address `bcrt1qqde3c4pmvrr9d3pav3v6hlpp9l3sm6rxnj8dcm`.

Below we'll decode the output addresses so that we know the output scriptPubkeys. For more on decoding addresses, refer back to the 'Addresses' chapter.

In [3]:
# Lisa's change output
lisa_change_value_sat = int(float("0.1") * 100000000)
lisa_change_spk = bech32_to_spk('bcrt', 'bcrt1qqde3c4pmvrr9d3pav3v6hlpp9l3sm6rxnj8dcm')

# Outputs to charities
output1_value_sat = int(float("0.2") * 100000000)
output1_spk = bech32_to_spk('bcrt', 'bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw')

output2_value_sat = int(float("0.2") * 100000000)
output2_spk = bech32_to_spk('bcrt', 'bcrt1q6mlqttg852e63uahyglwla55xusryqp08vx9w2')

output3_value_sat = int(float("0.2") * 100000000)
output3_spk = bech32_to_spk('bcrt', 'bcrt1qe9y40n9uwzh34mzj02w3xx9zkhgke6wxcql4lk')

### Create an unsigned transaction

As with the previous chapters, we'll fill in all the fields for our unsigned transaction but with our multiple inputs and outputs.

In [4]:
# VERSION
# version '2' indicates that we may use relative timelocks (BIP68)
version = bytes.fromhex("0200 0000")

# MARKER (new to segwit)
marker = bytes.fromhex("00")

# FLAG (new to segwit)
flag = bytes.fromhex("01")

# INPUTS
# We have 2 input
input_count = bytes.fromhex("02")

# Convert txid and index to bytes (little endian)
txid_a = (bytes.fromhex(txid_a))[::-1]
index_a = index_a.to_bytes(4, byteorder="little", signed=False)

txid_l = (bytes.fromhex(txid_l))[::-1]
index_l = index_l.to_bytes(4, byteorder="little", signed=False)


# For the unsigned transaction we use an empty scriptSig
scriptsig_a = bytes.fromhex("")
scriptsig_l = bytes.fromhex("")

# use 0xffffffff unless you are using OP_CHECKSEQUENCEVERIFY, locktime, or rbf
sequence = bytes.fromhex("ffff ffff")

inputs = (
    txid_a
    + index_a
    + varint_len(scriptsig_a)
    + scriptsig_a
    + sequence
    + txid_l
    + index_l
    + varint_len(scriptsig_l)
    + scriptsig_l
    + sequence
)

# OUTPUTS
# 0x02 for out two outputs
output_count = bytes.fromhex("04")

# Convert the values to 8 length byte arrays
output1_value = output1_value_sat.to_bytes(8, byteorder="little", signed=False)
output2_value = output2_value_sat.to_bytes(8, byteorder="little", signed=False)
output3_value = output3_value_sat.to_bytes(8, byteorder="little", signed=False)
lisa_change_value = lisa_change_value_sat.to_bytes(8, byteorder="little", signed=False)


outputs = (
    output1_value
    + varint_len(output1_spk)
    + output1_spk
    + output2_value
    + varint_len(output2_spk)
    + output2_spk
    + output3_value
    + varint_len(output3_spk)
    + output3_spk
    + lisa_change_value
    + varint_len(lisa_change_spk)
    + lisa_change_spk
)

# LOCKTIME
locktime = bytes.fromhex("0000 0000")

unsigned_tx = (
    version
    + input_count
    + inputs
    + output_count
    + outputs
    + locktime
)
print("unsigned_tx: ", unsigned_tx.hex())

unsigned_tx:  020000000246cb8bd3a2cf0fc9ff572488c7263118b6574c48c542c3b9ba8cce731a88d9960100000000ffffffffc9718a59b9cb3e662dc44c2c8ec1b411e3c554ec1246d5cd8fab380ead8eeebe0000000000ffffffff04002d310100000000160014fc7250a211deddc70ee5a2738de5f07817351cef002d310100000000160014d6fe05ad07a2b3a8f3b7223eeff694372032002f002d310100000000160014c94957ccbc70af1aec527a9d1318a2b5d16ce9c6809698000000000016001403731c543b60c656c43d6459abfc212fe30de86600000000


We can decode this raw transaction to inspect it and see that it has all the information we need apart from the segwit fields (version, marker, and witness).

In [5]:
decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + unsigned_tx.hex())
print(decoded)

{
  "txid": "2c21f2a55ddaf50e2a65dd652339528387362146d568a414aa47e13b22991af8",
  "hash": "2c21f2a55ddaf50e2a65dd652339528387362146d568a414aa47e13b22991af8",
  "version": 2,
  "size": 216,
  "vsize": 216,
  "weight": 864,
  "locktime": 0,
  "vin": [
    {
      "txid": "96d9881a73ce8cbab9c342c5484c57b6183126c7882457ffc90fcfa2d38bcb46",
      "vout": 1,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    },
    {
      "txid": "beee8ead0e38ab8fcdd54612ec54c5e311b4c18e2c4cc42d663ecbb9598a71c9",
      "vout": 0,
      "scriptSig": {
        "asm": "",
        "hex": ""
      },
      "sequence": 4294967295
    }
  ],
  "vout": [
    {
      "value": 0.20000000,
      "n": 0,
      "scriptPubKey": {
        "asm": "0 fc7250a211deddc70ee5a2738de5f07817351cef",
        "desc": "addr(bcrt1ql3e9pgs3mmwuwrh95fecme0s0qtn2880hlwwpw)#luvpmd0q",
        "hex": "0014fc7250a211deddc70ee5a2738de5f07817351cef",
        "address": "bcrt1ql3e9pgs3mmwuwrh95f

Segwit transactions have a new signing scheme described in [BIP143](https://github.com/bitcoin/bips/blob/master/bip-0143.mediawiki)


In [6]:
# pk_hash1 = hash160(pubkey1)
# scriptcode1 = bytes.fromhex("76a914" + pk_hash.hex() + "88ac")

# input_amount_sat = int(0.101 * 100_000_000)
# value = input_amount_sat.to_bytes(8, byteorder="little", signed=False)

# hashPrevOuts = hash256(txid_1 + index_1)
# hashSequence = hash256(sequence)
# hashOutputs = hash256(outputs)
# sighash_type = bytes.fromhex("0100 0000") # SIGHASH_ALL

# tx_digest_preimage = (
#     version
#     + hashPrevOuts
#     + hashSequence
#     + txid_1
#     + index_1
#     + varint_len(scriptcode1)
#     + scriptcode1
#     + value1
#     + sequence1
#     + hashOutputs
#     + locktime
#     + sighash_type
# )
# print(tx_digest_preimage.hex())

Now we are ready to hash this transaction and produce an ecdsa signature on it. 

Before hashing the transaction with hash256, we append the sighash flag. In this example we'll use the most commonly used SIGHASH_ALL flag, meaning the signature guarantees the input will only be used in a transaction with these exact inputs and outputs.

Note that when we append the sighash flag to the transaction, we use 4 bytes, however when we append the sighash flag to the end of the signature itself we only use 1 byte.

In [7]:
# # Create sigHash to be signed
# sighash = hash256(tx_digest_preimage)

# # Sign the sigHash with the input private key
# signing_key = ecdsa.SigningKey.from_string(sender_privkey, curve=ecdsa.SECP256k1) 
# signature = signing_key.sign_digest(sighash, sigencode=ecdsa.util.sigencode_der_canonize)

# # Append SIGHASH_ALL to the signature
# signature = signature + bytes.fromhex("01")

# # Witness field
# witness = (
#     # indicate the number of stack items for the txin
#     # 2 items for signature and pubkey
#     bytes.fromhex("02")
#     + pushbytes(signature)
#     + pushbytes(sender_pubkey)
# )

# # tx_in with our new sigScript containing the signature we just created
# inputs_signed = (
#     txid
#     + index
#     + varint_len(scriptsig)
#     + scriptsig
#     + sequence
# )

# # the final signed transaction
# signed_tx = (
#     version
#     + marker
#     + flag
#     + input_count
#     + inputs_signed
#     + output_count
#     + outputs
#     + witness
#     + locktime
# )

# print("signed transaction: ",signed_tx.hex())

### Broadcast the transaction (on regtest mode)
If we get back a txid (32 byte hash), then it means the tx was successfully broadcast! If we just want to see if the transaction would have been accepted, but without broadcasting it, we can use the `testmempoolaccept` command (commented out).

In [8]:
# new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest sendrawtransaction " + signed_tx.hex())
# # new_tx_txid = subprocess.getoutput("bitcoin-cli -regtest testmempoolaccept " + "'[\"" +  signed_tx.hex()+ "\"]'")

# print(new_tx_txid)

We can decode the serialized transaction using ```decoderawtransaction```. Notice that our output addresses match the change and receiver addresses from earlier.

In [9]:
# print("receiver's p2pkh address: " + receiver_address)
# change_p2pkh_addr = pk_to_p2pkh(change_pubkey, network = "regtest")
# print("sender's change p2pkh address: " + change_p2pkh_addr)

In [10]:
# decoded = subprocess.getoutput("bitcoin-cli -regtest decoderawtransaction " + signed_tx.hex())
# print(decoded)

In [11]:
# stop bitcoin core
subprocess.getoutput("bitcoin-cli -regtest stop")

'Bitcoin Core stopping'

## Quiz


 ## Answers
    

## Exercise
